# Make geojson for pulling MODIS data
- Read the site metadata generated in the previous step
- Calculate a 4km x 4km bounding box in WGS84
- Ensure time format is compatible with colab script

In [5]:
import pandas as pd
from shapely.geometry import Polygon
from pyproj import Proj, transform, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import geopandas as gpd
from pathlib import Path

In [6]:
META_DIR = Path('data/meta')
meta = pd.read_csv(META_DIR / 'processed_site_meta.csv')
meta

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,IGBP,TIME_INFO
0,BE-Bra,51.307610,4.519840,16.0,ENF,"{'icos-2023,icos-ww,fluxnet': ['1996_01_01', '..."
1,BE-Dor,50.311874,4.968113,253.0,GRA,"{'icos-ww': ['2011_01_01', '2020_12_31']}"
2,BE-Lon,50.551620,4.746234,170.0,CRO,"{'icos-2023,icos-ww,fluxnet': ['2004_01_01', '..."
3,BE-Maa,50.979870,5.631851,87.0,CSH,"{'icos-2023,icos-ww': ['2016_01_01', '2022_12_..."
4,BE-Vie,50.304962,5.998099,490.0,MF,"{'icos-2023,icos-ww,fluxnet': ['1996_01_01', '..."
...,...,...,...,...,...,...
381,RU-Ha1,54.725200,90.002200,446.0,GRA,"{'fluxnet': ['2002_01_01', '2004_12_31']}"
382,SD-Dem,13.282900,30.478300,500.0,SAV,"{'fluxnet': ['2005_01_01', '2009_12_31']}"
383,SJ-Adv,78.186000,15.923000,17.0,WET,"{'fluxnet': ['2011_01_01', '2014_12_31']}"
384,SN-Dhr,15.402800,-15.432200,40.0,SAV,"{'fluxnet': ['2010_01_01', '2013_12_31']}"


In [7]:
def get_4km_polygon_wkt(lat, lon):
    utm_crs_info = query_utm_crs_info(
        area_of_interest=AreaOfInterest(west_lon_degree=lon, south_lat_degree=lat, east_lon_degree=lon, north_lat_degree=lat),
        datum_name="WGS 84"
    )[0]
    utm_crs = CRS.from_epsg(utm_crs_info.code)

    # Projected coordinate system for accurate distance measurement
    proj_utm = Proj(utm_crs)

    # Convert the given lat/lon in WGS84 to UTM coordinates
    x, y = proj_utm(lon, lat)

    # Define the offsets (2km in each direction to form a 4km box)
    offset = 2000  # meters

    # Calculate the corners of the box in UTM coordinates
    bottom_left = (x - offset, y - offset)
    bottom_right = (x + offset, y - offset)
    top_right = (x + offset, y + offset)
    top_left = (x - offset, y + offset)

    # Create a polygon from these corners
    box = Polygon([bottom_left, bottom_right, top_right, top_left, bottom_left])

    # Optionally, convert the polygon back to geographic coordinates (WGS84)
    proj_wgs84 = Proj(proj='latlong', datum='WGS84')
    lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)

    # Create a new polygon in geographic coordinates
    geo_polygon = Polygon(zip(lon_lat_polygon[0], lon_lat_polygon[1]))

    return geo_polygon

meta['geometry'] = meta.apply(lambda row: get_4km_polygon_wkt(row['LOCATION_LAT'], row['LOCATION_LON']), axis=1)
meta

/tmp/ipykernel_4300/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_4300/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_4300/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_4300/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_4300/

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,IGBP,TIME_INFO,geometry
0,BE-Bra,51.307610,4.519840,16.0,ENF,"{'icos-2023,icos-ww,fluxnet': ['1996_01_01', '...","POLYGON ((4.490574806448662 51.29000080623209,..."
1,BE-Dor,50.311874,4.968113,253.0,GRA,"{'icos-ww': ['2011_01_01', '2020_12_31']}","POLYGON ((4.939308871234061 50.29436913709019,..."
2,BE-Lon,50.551620,4.746234,170.0,CRO,"{'icos-2023,icos-ww,fluxnet': ['2004_01_01', '...","POLYGON ((4.717362385562432 50.53406149464479,..."
3,BE-Maa,50.979870,5.631851,87.0,CSH,"{'icos-2023,icos-ww': ['2016_01_01', '2022_12_...","POLYGON ((5.602385608194751 50.9625416417985, ..."
4,BE-Vie,50.304962,5.998099,490.0,MF,"{'icos-2023,icos-ww,fluxnet': ['1996_01_01', '...","POLYGON ((5.968932875475469 50.28771950500013,..."
...,...,...,...,...,...,...,...
381,RU-Ha1,54.725200,90.002200,446.0,GRA,"{'fluxnet': ['2002_01_01', '2004_12_31']}",POLYGON ((89.97252852708998 54.706480070724574...
382,SD-Dem,13.282900,30.478300,500.0,SAV,"{'fluxnet': ['2005_01_01', '2009_12_31']}",POLYGON ((30.46004171921137 13.264649338969441...
383,SJ-Adv,78.186000,15.923000,17.0,WET,"{'fluxnet': ['2011_01_01', '2014_12_31']}",POLYGON ((15.834255224173658 78.16835024107449...
384,SN-Dhr,15.402800,-15.432200,40.0,SAV,"{'fluxnet': ['2010_01_01', '2013_12_31']}",POLYGON ((-15.450799221973599 15.3846817291181...


In [8]:
gdf = gpd.GeoDataFrame(meta, crs='EPSG:4326', geometry=meta['geometry'])
gdf.to_file(META_DIR / 'sites.geojson', driver='GeoJSON')